In [1]:
import cv2
import numpy as np
import mediapipe as mp
import math

# Initialize mediapipe hand detection
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Calculator variables
expression = ""
result = ""

# Button positions and labels
buttons = {
    '1': (50, 150, 100, 200),
    '2': (110, 150, 160, 200),
    '3': (170, 150, 220, 200),
    '4': (50, 210, 100, 260),
    '5': (110, 210, 160, 260),
    '6': (170, 210, 220, 260),
    '7': (50, 270, 100, 320),
    '8': (110, 270, 160, 320),
    '9': (170, 270, 220, 320),
    '0': (110, 330, 160, 380),
    '+': (230, 150, 280, 200),
    '-': (230, 210, 280, 260),
    '*': (230, 270, 280, 320),
    '/': (230, 330, 280, 380),
    '=': (290, 330, 340, 380),
    'C': (50, 330, 100, 380)
}

# Draw calculator buttons on the frame
def draw_buttons(frame):
    for key, pos in buttons.items():
        x1, y1, x2, y2 = pos
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), -1)
        cv2.putText(frame, key, (x1 + 15, y1 + 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

# Check if a point is inside a button
def is_inside_button(center, button_pos):
    x, y = center
    x1, y1, x2, y2 = button_pos
    return x1 <= x <= x2 and y1 <= y <= y2

# Evaluate the calculator expression
def calculate_expression(expression):
    try:
        return str(eval(expression))
    except:
        return "Error"

# Calculate the distance between two points
def distance(p1, p2):
    return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

# Debounce state variable
is_pressed = False  # Keeps track if a button is currently pressed

# Main loop
while True:
    # Read a frame from the webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Flip and process the frame
    frame = cv2.flip(frame, 1)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Draw the calculator buttons
    draw_buttons(frame)

    # Display the current expression and result
    cv2.putText(frame, "Expr: " + expression, (10, 100),  cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.putText(frame, "Result: " + result, (10, 130),  cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Get hand landmark prediction
    result_hands = hands.process(framergb)

    # If hand landmarks are detected
    if result_hands.multi_hand_landmarks:
        for hand_landmarks in result_hands.multi_hand_landmarks:
            # Get the tip of the index finger (landmark 8) and thumb tip (landmark 4)
            index_finger_tip = hand_landmarks.landmark[8]
            thumb_tip = hand_landmarks.landmark[4]

            h, w, c = frame.shape
            index_x, index_y = int(index_finger_tip.x * w), int(index_finger_tip.y * h)
            thumb_x, thumb_y = int(thumb_tip.x * w), int(thumb_tip.y * h)

            # Draw circles on the tips of the index finger and thumb
            cv2.circle(frame, (index_x, index_y), 10, (255, 0, 255), cv2.FILLED)
            cv2.circle(frame, (thumb_x, thumb_y), 10, (255, 0, 0), cv2.FILLED)

            # Calculate the distance between the thumb and index finger
            dist = distance((index_x, index_y), (thumb_x, thumb_y))

            # Detect finger proximity and ensure only one selection is made when pressed
            if dist < 30:  # Fingers are close together, indicating a press
                if not is_pressed:  # Only process if this is a new press
                    is_pressed = True  # Mark as pressed

                    # Check if the finger is pressing a button
                    for key, pos in buttons.items():
                        if is_inside_button((index_x, index_y), pos):
                            if key == '=':
                                result = calculate_expression(expression)
                                expression = ""  # Clear the expression after evaluation
                            elif key == 'C':
                                expression = ""
                                result = ""
                            else:
                                expression += key
                            # Break to avoid multiple detections in one frame
                            break

            else:
                is_pressed = False  # Reset the press flag when fingers are apart

            # Draw hand landmarks
            mpDraw.draw_landmarks(frame, hand_landmarks, mpHands.HAND_CONNECTIONS)

    # Display the frame
    cv2.imshow("Hand Gesture Calculator", frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()